In [1]:
import math
import os
import random
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.benchmark = True  # Автопоиск быстрых алгоритмов
    torch.backends.cudnn.deterministic = False  # Для скорости, но немного жертвуем воспроизводимостью

set_seed(42)

# ArcFace с возможностью менять margin
class ArcFace(nn.Module):
    def __init__(self, in_features, out_features, s=64.0, m=0.5, eps=1e-7):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = float(s)
        self.m = float(m)
        self.eps = eps

        # Веса классификатора (центры классов)
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        #Используем равномерное распределение, диапазон которого зависит от числа входящих и выходящих нейронов
        nn.init.xavier_uniform_(self.weight)

        # Вычисляем тригонометрические константы
        self.cos_m = math.cos(self.m)
        self.sin_m = math.sin(self.m)

    def set_margin(self, m: float):
        #Изменяет коэффициент m и пересчитывает тригонометрические константы
        self.m = float(m)
        self.cos_m = math.cos(self.m)
        self.sin_m = math.sin(self.m)

    def forward(self, embeddings, labels=None):
        # Нормализуем эмбеддинги и веса
        x = F.normalize(embeddings, p=2, dim=1)  # (B, D) - размер батча и размерность эмбеддинга, p=2 -> L2 нормализация
        W = F.normalize(self.weight, p=2, dim=1)  # (N, D) - количество классов и размерность эмбеддинга
        
        # Вычисляем косинусы углов между эмбеддингами и центрами классов, clamp ограничивает снизу нулём, чтобы вдруг не произошло округления к отрицательному числу
        cosine = F.linear(x, W)  # (B, N)
        cosine = cosine.clamp(-1.0 + self.eps, 1.0 - self.eps)
        
        # Если меток нет (инференс), просто возвращает масштабированные косинусы
        if labels is None:
            return self.s * cosine
        
        # Для обучения вычисляем sin(theta)
        sine = torch.sqrt(torch.clamp(1.0 - cosine * cosine, min=0.0) + self.eps)
        
        # Вычисляем cos(theta + m) используя тригонометрическую формулу косинуса суммы
        cos_theta_m = cosine * self.cos_m - sine * self.sin_m
        
        # Создаем one-hot вектор для целевых классов
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1, 1).long(), 1.0)
        
        # Формируем финальные логиты:
        # Для целевого класса используем cos(theta + m)
        # Для остальных классов используем cos(theta)
        logits = cosine.clone()
        logits = logits * (1.0 - one_hot) + cos_theta_m * one_hot
        
        # Масштабируем логиты
        logits = logits * self.s
        
        return logits

class FaceModel(nn.Module):
    def __init__(
        self,
        num_classes,
        embedding_size=512,
        use_arcface=False,
        arc_s=64.0,
        arc_m=0.55,
    ):
        super().__init__()
        self.use_arcface = bool(use_arcface)

        weights = models.ResNet34_Weights.IMAGENET1K_V1
        backbone = models.resnet34(weights=weights)
        in_features = backbone.fc.in_features
        backbone.fc = nn.Identity()
        self.backbone = backbone

        # Изначально все слои замораживаю - разморожу layers 3 и 4 на втором этапе обучения
        for p in self.backbone.parameters():
            p.requires_grad = False

        self.embedding = nn.Sequential(
            nn.Linear(in_features, embedding_size),
            nn.BatchNorm1d(embedding_size),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.3)
        )

        if self.use_arcface:
            self.classifier = ArcFace(in_features=embedding_size, out_features=num_classes, s=arc_s, m=arc_m)
        else:
            self.classifier = nn.Linear(embedding_size, num_classes)
            # Тут используем такую же инициализацию весов, как в arcface
            nn.init.xavier_uniform_(self.classifier.weight)
            if self.classifier.bias is not None:
                nn.init.constant_(self.classifier.bias, 0.0)

    def forward(self, x, labels=None):
        feats = self.backbone(x)
        embeddings = self.embedding(feats)
        
        if self.use_arcface:
            embeddings = F.normalize(embeddings, p=2, dim=1)
            logits_or_cos = self.classifier(embeddings, labels)
            return logits_or_cos, embeddings
        else:
            logits = self.classifier(embeddings)
            embeddings = F.normalize(embeddings, p=2, dim=1)
            return logits, embeddings

# Подготовка данных
print("\n   Загрузка данных")

train_df = pd.read_csv(r'datasets/train_aligned.csv')
val_df = pd.read_csv(r'datasets/val_aligned.csv')
test_df = pd.read_csv(r'datasets/test_aligned.csv')

print(f"Train: {len(train_df)} записей")
print(f"Val: {len(val_df)} записей")
print(f"Test: {len(test_df)} записей")

# Поскольку id людей в отобранном датасете идут не по порядку, применяю Label Encoding
label_encoder = LabelEncoder()
all_labels = pd.concat([train_df['person'], val_df['person'], test_df['person']])
label_encoder.fit(all_labels)

train_df['encoded_person'] = label_encoder.transform(train_df['person'])
val_df['encoded_person'] = label_encoder.transform(val_df['person'])
test_df['encoded_person'] = label_encoder.transform(test_df['person'])

NUM_CLASSES = len(label_encoder.classes_)
print(f"Количество классов: {NUM_CLASSES}")

'''Миксап тоже представляет из себя технику аугментации, проводящую линейную интерполяцию между двумя изображениями и их метками.
   Если простым языком - попиксельно соединяет две фотографии в одну, вместо конкретной метки [1, 0] тоже получаем [X, 1-X]'''
def mixup_data(x, y, alpha=0.25):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size, device=x.device)
    
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    
    return mixed_x, y_a, y_b, lam

train_transform = A.Compose([
    '''Зеркальное отражение хоть и является геометрической аугментацией, но расположение глаз всё таки не меняет, поэтому использую её.
       Остальные аугментации, затрагивающие геометрию изображений, вероятно будут ухудшать результат - иначе зачем тогда на прошлом этапе мы
       проводили выравнивание?'''
    A.HorizontalFlip(p=0.5),
    
    A.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.15,
        p=0.8
    ),
    A.ToGray(p=0.2),
    A.GaussianBlur(blur_limit=(3, 3), p=0.33),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])

class FaceDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform
        
        '''В попытке перестроить структуру проекта на более-менее внятно разложенные по папкам файлы, в прошлом задании
           написал некорректные пути доступа к фотографиям. Иправить несложно конечно, но осадочек остался'''
        self.data['path'] = self.data['path'].apply(lambda x: x[3:])
        self.data['path'] = self.data['path'].replace(r'gavri/datasets', r'gavri/FRProject/datasets')
        
        self.paths = self.data['path'].values.tolist()
        self.labels = self.data['encoded_person'].values.tolist()
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path = self.paths[idx]
        
        # OpenCV
        image = cv2.imread(file_path)
        if image is None:
            # Fallback
            image = np.ones((112, 112, 3), dtype=np.uint8) * 255
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Применяем Albumentations трансформации
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return image, label


train_dataset = FaceDataset(train_df, train_transform)
val_dataset = FaceDataset(val_df, val_transform)
test_dataset = FaceDataset(test_df, val_transform)


BATCH_SIZE = 32  # на практике оказалось самым рабочим размером


print(f"\nИспользуем batch_size: {BATCH_SIZE}")
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False)


def train_epoch(model, loader, optimizer, criterion, use_mixup=False, mixup_alpha=0.25):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        # Если ArcFace - НЕ используем MixUp
        if getattr(model, 'use_arcface', False):
            logits, _ = model(images, labels)
            loss = criterion(logits, labels)
            _, predicted = logits.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        # Если обычный классификатор + включен MixUp
        elif use_mixup and np.random.random() < 0.5:
            mixed_images, labels_a, labels_b, lam = mixup_data(images, labels, alpha=mixup_alpha)
            logits, _ = model(mixed_images)
            
            # Смешанный лосс для MixUp
            loss = lam * criterion(logits, labels_a) + (1 - lam) * criterion(logits, labels_b)
            
            # Смешанная accuracy (для мониторинга)
            with torch.no_grad():
                _, predicted = logits.max(1)
                total += labels.size(0)
                # Weighted accuracy для MixUp
                correct_mixup = lam * (predicted == labels_a).float() + (1 - lam) * (predicted == labels_b).float()
                correct += correct_mixup.sum().item()
        
        # Обычный режим (без MixUp)
        else:
            logits, _ = model(images)
            loss = criterion(logits, labels)
            _, predicted = logits.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(loader)
    acc = correct / total if total > 0 else 0.0
    return avg_loss, acc


def validate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            logits, _ = model(images)  # labels=None - arcface вернет S*cos без маржи (не будет усложнять, как он это делает на обучении)
            preds = torch.argmax(logits, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return accuracy_score(all_labels, all_preds)

Используемое устройство: cuda

   Загрузка данных
Train: 19028 записей
Val: 2378 записей
Test: 2379 записей
Количество классов: 1200

Используем batch_size: 32


Обучение с CE Loss

In [3]:
print("\nСоздание модели ResNet34 (без ArcFace)")
model = FaceModel(num_classes=NUM_CLASSES, embedding_size=512, use_arcface=False).to(device)

#label_smoothing=0.3 помогает модели меньше перобучаться за счет небольшого распределения уверенности модели на другие классы
criterion = nn.CrossEntropyLoss(label_smoothing=0.3)


best_val_acc = 0.0
best_checkpoint = None


print("\nЭтап 1: Только head (15 эпох)")

for p in model.embedding.parameters():
    p.requires_grad = True
for p in model.classifier.parameters():
    p.requires_grad = True

optimizer_stage1 = optim.AdamW(
    [
        {'params': model.embedding.parameters(), 'lr': 8e-4},
        {'params': model.classifier.parameters(), 'lr': 8e-4}
    ],
    weight_decay=0.03
)

scheduler_stage1 = optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer_stage1, 
    T_0=5,           # Первый цикл - 5 эпох
    T_mult=2,        # Удваиваем длину каждого следующего цикла
    eta_min=1e-6     # Минимальный LR
)

EPOCHS_STAGE1 = 15
for epoch in range(EPOCHS_STAGE1):
    print(f"\nЭпоха {epoch+1}/{EPOCHS_STAGE1}")

    train_loss, train_acc = train_epoch(model, train_loader, optimizer_stage1, criterion, use_mixup=True)
    val_acc = validate(model, val_loader)
    scheduler_stage1.step()

    print(f"Train: loss={train_loss:.4f}, acc={train_acc:.4f}")
    print(f"Val: acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_checkpoint = {
            'stage': 'stage1',
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'train_acc': train_acc,
            'val_acc': val_acc
        }
        torch.save(best_checkpoint, r'weights\best_stage1_ce.pth')
        print(f"Сохранена лучшая модель (val_acc={val_acc:.4f})")


print("\nЭтап 2: Размораживаем весь backbone (35 эпох)")

for param in model.backbone.layer3.parameters():
    param.requires_grad = True
for param in model.backbone.layer4.parameters():
    param.requires_grad = True

optimizer_stage2 = optim.AdamW(
    [
        {'params': model.backbone.layer3.parameters(), 'lr': 2e-4},
        {'params': model.backbone.layer4.parameters(), 'lr': 3e-4},
        {'params': model.embedding.parameters(), 'lr': 4e-4},
        {'params': model.classifier.parameters(), 'lr': 4e-4}
    ],
    weight_decay=0.05
)

patience = 7
patience_counter = 0
EPOCHS_STAGE2 = 35

scheduler_stage2 = optim.lr_scheduler.CosineAnnealingLR(
    optimizer_stage2,
    T_max=EPOCHS_STAGE2,  # 35 эпох плавного снижения
    eta_min=1e-7
)

for epoch in range(EPOCHS_STAGE2):
    print(f"\nЭпоха {epoch+1}/{EPOCHS_STAGE2}")

    train_loss, train_acc = train_epoch(model, train_loader, optimizer_stage2, criterion, use_mixup=True)
    val_acc = validate(model, val_loader)
    scheduler_stage2.step()

    print(f"Train: loss={train_loss:.4f}, acc={train_acc:.4f}")
    print(f"Val: acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        best_checkpoint = {
            'stage': 'stage2',
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'train_acc': train_acc,
            'val_acc': val_acc
        }
        torch.save(best_checkpoint, r'weights\best_stage2_ce.pth')
        print(f"Сохранена лучшая модель (val_acc={val_acc:.4f})")
    else:
        patience_counter += 1
        print(f"  Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("Ранняя остановка stage2")
            break


print("\nФинальный fine-tune")
optimizer_finetune = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=5e-6,
    weight_decay=0.001
)
scheduler_finetune = optim.lr_scheduler.ReduceLROnPlateau(optimizer_finetune, mode='max', factor=0.5, patience=1)

patience = 5
patience_counter = 0
EPOCHS_FINETUNE = 15

for epoch in range(EPOCHS_FINETUNE):
    print(f"\nЭпоха {epoch+1}/{EPOCHS_FINETUNE}")

    train_loss, train_acc = train_epoch(model, train_loader, optimizer_finetune, criterion, use_mixup=True)
    val_acc = validate(model, val_loader)
    scheduler_finetune.step(val_acc)

    print(f"Train: loss={train_loss:.4f}, acc={train_acc:.4f}")
    print(f"Val: acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        best_checkpoint = {
            'stage': 'finetune',
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'train_acc': train_acc,
            'val_acc': val_acc
        }
        torch.save(best_checkpoint, r'weights\best_final_ce.pth')
        print(f"Сохранена лучшая модель (val_acc={val_acc:.4f})")
    else:
        patience_counter += 1
        print(f"  Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("Ранняя остановка finetune")
            break
            
print(f"\nЛучшая точность на валидации (best_val_acc): {best_val_acc:.4f}")

print("\n   Загрузка лучшей модели")
best_model_path = None
if os.path.exists(r'weights\best_final_ce.pth'):
    best_model_path = r'weights\best_final_ce.pth'
elif os.path.exists(r'weights\best_stage2_ce.pth'):
    best_model_path = r'weights\best_stage2_ce.pth'

checkpoint = torch.load(best_model_path, map_location=device, weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Загружена лучшая модель из {best_model_path}")
print(f"  Val accuracy (saved): {checkpoint.get('val_acc', 0):.4f}")
print(f"  Train accuracy (saved): {checkpoint.get('train_acc', 0):.4f}")

print("\n   Тестирование")
test_acc = validate(model, test_loader)
print(f"Точность на тестовом наборе: {test_acc:.4f}")

print("\n   Сохранение финальной модели")
final_checkpoint = {
    'model_state_dict': model.state_dict(),
    'val_acc': best_val_acc,
    'test_acc': test_acc,
    'train_acc': best_checkpoint.get('train_acc', 0) if best_checkpoint is not None else 0,
    'num_classes': NUM_CLASSES,
    'model_type': 'FaceModel_CE'
}
torch.save(final_checkpoint, r'weights\final_ce_model.pth')
print(r"Финальная модель сохранена в 'weights\final_ce_model.pth'")

print(f"\n{'='*60}")
print("ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ")
print('='*60)
print(f"Лучшая val accuracy: {best_val_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")


Создание модели ResNet34 (без ArcFace)

Этап 1: Только head (15 эпох)

Эпоха 1/15
Train: loss=7.0013, acc=0.0169
Val: acc=0.0429
Сохранена лучшая модель (val_acc=0.0429)

Эпоха 2/15
Train: loss=6.3799, acc=0.0734
Val: acc=0.0887
Сохранена лучшая модель (val_acc=0.0887)

Эпоха 3/15
Train: loss=6.1109, acc=0.1276
Val: acc=0.1253
Сохранена лучшая модель (val_acc=0.1253)

Эпоха 4/15
Train: loss=5.9291, acc=0.1713
Val: acc=0.1417
Сохранена лучшая модель (val_acc=0.1417)

Эпоха 5/15
Train: loss=5.8126, acc=0.2075
Val: acc=0.1451
Сохранена лучшая модель (val_acc=0.1451)

Эпоха 6/15
Train: loss=5.9308, acc=0.1674
Val: acc=0.1333

Эпоха 7/15
Train: loss=5.8213, acc=0.1944
Val: acc=0.1556
Сохранена лучшая модель (val_acc=0.1556)

Эпоха 8/15
Train: loss=5.7617, acc=0.2116
Val: acc=0.1699
Сохранена лучшая модель (val_acc=0.1699)

Эпоха 9/15
Train: loss=5.6374, acc=0.2458
Val: acc=0.1901
Сохранена лучшая модель (val_acc=0.1901)

Эпоха 10/15
Train: loss=5.5244, acc=0.2786
Val: acc=0.1892

Эпоха 11/

Обучение с ArcFaceLoss

In [4]:
print("\nСоздание модели ResNet34 (с ArcFace)")
model = FaceModel(num_classes=NUM_CLASSES, embedding_size=512, use_arcface=True, arc_s=64.0, arc_m=0.0).to(device)

# При ArcFace НЕ используем label_smoothing
criterion = nn.CrossEntropyLoss()


best_val_acc = 0.0
best_checkpoint = None

print("\nЭтап 1: Только head (15 эпох)")

for p in model.embedding.parameters():
    p.requires_grad = True
for p in model.classifier.parameters():
    p.requires_grad = True

optimizer_stage1 = optim.AdamW(
    [
        {'params': model.embedding.parameters(), 'lr': 8e-4},
        {'params': model.classifier.parameters(), 'lr': 8e-4}
    ],
    weight_decay=0.03
)

scheduler_stage1 = optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer_stage1, 
    T_0=5,           # Первый цикл - 5 эпох
    T_mult=2,        # Удваиваем длину каждого следующего цикла
    eta_min=1e-6     # Минимальный LR
)

EPOCHS_STAGE1 = 15
for epoch in range(EPOCHS_STAGE1):
    print(f"\nЭпоха {epoch+1}/{EPOCHS_STAGE1}")
    current_margin = 0.35 * (epoch / (EPOCHS_STAGE1 - 1)) if EPOCHS_STAGE1 > 1 else 0.0
    model.classifier.set_margin(current_margin)
    print(f"  m изменен на: {current_margin:.4f}")

    train_loss, train_acc = train_epoch(model, train_loader, optimizer_stage1, criterion, use_mixup=False)
    val_acc = validate(model, val_loader)
    scheduler_stage1.step()

    print(f"Train: loss={train_loss:.4f}, acc={train_acc:.4f}")
    print(f"Val: acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_checkpoint = {
            'stage': 'stage1',
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'train_acc': train_acc,
            'val_acc': val_acc
        }
        torch.save(best_checkpoint, r'weights\best_stage1_arc.pth')
        print(f"Сохранена лучшая модель (val_acc={val_acc:.4f})")



print("\nЭтап 2: Размораживаем весь backbone (35 эпох)")

for param in model.backbone.layer3.parameters():
    param.requires_grad = True
for param in model.backbone.layer4.parameters():
    param.requires_grad = True

optimizer_stage2 = optim.AdamW(
    [
        {'params': model.backbone.layer3.parameters(), 'lr': 2e-4},
        {'params': model.backbone.layer4.parameters(), 'lr': 3e-4},
        {'params': model.embedding.parameters(), 'lr': 4e-4},
        {'params': model.classifier.parameters(), 'lr': 4e-4}
    ],
    weight_decay=0.05
)

patience = 7
patience_counter = 0
EPOCHS_STAGE2 = 35

scheduler_stage2 = optim.lr_scheduler.CosineAnnealingLR(
    optimizer_stage2,
    T_max=EPOCHS_STAGE2,  # 35 эпох плавного снижения
    eta_min=1e-7
)

for epoch in range(EPOCHS_STAGE2):
    print(f"\nЭпоха {epoch+1}/{EPOCHS_STAGE2}")
    current_margin = (0.1 + 0.45 * (epoch / 15)) if epoch < 15  else 0.55
    if epoch < 16:
        model.classifier.set_margin(current_margin)
        print(f"  Margin m set to: {current_margin:.4f}")

    train_loss, train_acc = train_epoch(model, train_loader, optimizer_stage2, criterion, use_mixup=False)
    val_acc = validate(model, val_loader)
    scheduler_stage2.step()

    print(f"Train: loss={train_loss:.4f}, acc={train_acc:.4f}")
    print(f"Val: acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        best_checkpoint = {
            'stage': 'stage2',
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'train_acc': train_acc,
            'val_acc': val_acc
        }
        torch.save(best_checkpoint, r'weights\best_stage2_arc.pth')
        print(f"Сохранена лучшая модель (val_acc={val_acc:.4f})")
    else:
        patience_counter += 1
        print(f"  Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("Ранняя остановка stage2")
            break


print("\nФинальный fine-tune")
optimizer_finetune = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=5e-6,
    weight_decay=0.001
)
scheduler_finetune = optim.lr_scheduler.ReduceLROnPlateau(optimizer_finetune, mode='max', factor=0.5, patience=1)

patience = 5
patience_counter = 0
EPOCHS_FINETUNE = 15

current_margin = 0.5
model.classifier.set_margin(current_margin)
print(f"m изменён на: {current_margin:.4f}")

for epoch in range(EPOCHS_FINETUNE):
    print(f"\nЭпоха {epoch+1}/{EPOCHS_FINETUNE}")

    train_loss, train_acc = train_epoch(model, train_loader, optimizer_finetune, criterion, use_mixup=False)
    val_acc = validate(model, val_loader)
    scheduler_finetune.step(val_acc)

    print(f"Train: loss={train_loss:.4f}, acc={train_acc:.4f}")
    print(f"Val: acc={val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        best_checkpoint = {
            'stage': 'finetune',
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'train_acc': train_acc,
            'val_acc': val_acc
        }
        torch.save(best_checkpoint, r'weights\best_final_arc.pth')
        print(f"Сохранена лучшая модель (val_acc={val_acc:.4f})")
    else:
        patience_counter += 1
        print(f"  Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("Ранняя остановка finetune")
            break
            
print(f"\nЛучшая точность на валидации (best_val_acc): {best_val_acc:.4f}")


print("\n   Загрузка лучшей модели")
best_model_path = None
if os.path.exists(r'weights\best_final_arc.pth'):
    best_model_path = r'weights\best_final_arc.pth'
elif os.path.exists(r'weights\best_stage2_arc.pth'):
    best_model_path = r'weights\best_stage2_arc.pth'

checkpoint = torch.load(best_model_path, map_location=device, weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Загружена лучшая модель из {best_model_path}")
print(f"  Val accuracy (saved): {checkpoint.get('val_acc', 0):.4f}")
print(f"  Train accuracy (saved): {checkpoint.get('train_acc', 0):.4f}")


print("\n   Тестирование")
test_acc = validate(model, test_loader)
print(f"Точность на тестовом наборе: {test_acc:.4f}")


print("\n   Сохранение финальной модели")
final_checkpoint = {
    'model_state_dict': model.state_dict(),
    'val_acc': best_val_acc,
    'test_acc': test_acc,
    'train_acc': best_checkpoint.get('train_acc', 0) if best_checkpoint is not None else 0,
    'num_classes': NUM_CLASSES,
    'model_type': 'FaceModel_ArcFace'
}
torch.save(final_checkpoint, r'weights\final_arc_model.pth')
print(r"Финальная модель сохранена в 'weights\final_arc_model.pth'")


print(f"\n{'='*60}")
print("ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ")
print('='*60)
print(f"Лучшая val accuracy: {best_val_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")


Создание модели ResNet34 (с ArcFace)

Этап 1: Только head (15 эпох)

Эпоха 1/15
  m изменен на: 0.0000
Train: loss=8.0995, acc=0.0152
Val: acc=0.0547
Сохранена лучшая модель (val_acc=0.0547)

Эпоха 2/15
  m изменен на: 0.0250
Train: loss=7.1664, acc=0.0223
Val: acc=0.1001
Сохранена лучшая модель (val_acc=0.1001)

Эпоха 3/15
  m изменен на: 0.0500
Train: loss=7.7232, acc=0.0125
Val: acc=0.1434
Сохранена лучшая модель (val_acc=0.1434)

Эпоха 4/15
  m изменен на: 0.0750
Train: loss=8.6002, acc=0.0063
Val: acc=0.1770
Сохранена лучшая модель (val_acc=0.1770)

Эпоха 5/15
  m изменен на: 0.1000
Train: loss=9.7070, acc=0.0019
Val: acc=0.1808
Сохранена лучшая модель (val_acc=0.1808)

Эпоха 6/15
  m изменен на: 0.1250
Train: loss=11.7216, acc=0.0001
Val: acc=0.1632

Эпоха 7/15
  m изменен на: 0.1500
Train: loss=12.9627, acc=0.0004
Val: acc=0.1678

Эпоха 8/15
  m изменен на: 0.1750
Train: loss=14.1199, acc=0.0001
Val: acc=0.1918
Сохранена лучшая модель (val_acc=0.1918)

Эпоха 9/15
  m изменен на

В обоих случаях разбивал обучение на 3 фазы:
1) Только embedding и classifier слои, не изменяя предобученных на imagenet весов бэкбона, чтобы хоть немного приблизиться на новых слоях к оптимальным значениям весов и они не так сильно влияли на первых эпохах на градиенты предобученных слоев
2) Разморозил два слоя backbonе, обучал с помощью косинусного шедулера, который плавно понижает LR в зависимости от эпохи
3) Не добавляя новых слоев пытался дообучить модель с изначально низким lr и его понижением на плато с заведомо низкой терпимостью всего в одну эпоху. На последней итерации обучения смысла от этого не оказалось, однако на прошлых давало прирост вплоть до 1%

Что же касается особенностей обучения arcface. При попытках запустить обучение сразу с m = 0.5 модель не обучалась (как оказалось тогда у меня была некорректная валидация, на которой я не убирал m), вследствие чего было принято решение попробовать "разогревать" m, постепенно поднимая его от эпохи к эпохе и в определенный момент фиксируя на определенном значении (0.5). К концу такого обучения модель достигла вероятности правильной классификации 65% даже с условием маржи, равной 0.5. Так же при обучении с ним нельзя было использовать MixUp и Label Smoothing, как я это сделал при обучении на CE. Mixup не используется по причине того, его целью является создание плавного перехода между лицами, в то время как arcface наоборот направлен на максимизацию углового расстояния между ними. LabelSmoothing же нельзя использовать, потому что arcface целенаправленно штрафует правильную метку, что при ее размытии будет работать некорректно.